In [1]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read churn data into DataFrame using pandas
import pandas as pd

df = pd.read_csv('source_output/churn_clean.csv', header='infer')

In [3]:
# select relevant variables
df = df[['Area',
        'Children',
        'Age',
        'Income',
        'Marital',
        'Gender',
        'Email',
        'Contract',
        'Port_modem',
        'Tablet',
        'InternetService',
        'Phone',
        'Multiple',
        'OnlineSecurity',
        'OnlineBackup',
        'DeviceProtection',
        'TechSupport',
        'StreamingTV',
        'StreamingMovies',
        'PaperlessBilling',
        'PaymentMethod',
        'Tenure',
        'MonthlyCharge',
        'Bandwidth_GB_Year']]

In [4]:
df.head()

,Area,Children,Age,Income,Marital,Gender,Email,Contract,Port_modem,Tablet,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,PaymentMethod,Tenure,MonthlyCharge,Bandwidth_GB_Year
0,Urban,0,68,28561.99,Widowed,Male,10,One year,Yes,Yes,...,Yes,No,No,No,Yes,Yes,Credit Card (automatic),6.795513,172.455519,904.536110
1,Urban,1,27,21704.77,Married,Female,12,Month-to-month,No,Yes,...,No,No,No,Yes,Yes,Yes,Bank Transfer(automatic),1.156681,242.632554,800.982766
2,Urban,4,50,9609.57,Widowed,Female,9,Two Year,Yes,No,...,No,No,No,No,Yes,Yes,Credit Card (automatic),15.754144,159.947583,2054.706961
3,Suburban,1,48,18925.23,Married,Male,15,Two Year,No,No,...,No,No,No,Yes,No,Yes,Mailed Check,17.087227,119.956840,2164.579412
4,Suburban,0,83,40074.19,Separated,Male,16,Month-to-month,Yes,No,...,No,No,Yes,Yes,No,No,Mailed Check,1.670972,149.948316,271.493436


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               10000 non-null  object 
 1   Children           10000 non-null  int64  
 2   Age                10000 non-null  int64  
 3   Income             10000 non-null  float64
 4   Marital            10000 non-null  object 
 5   Gender             10000 non-null  object 
 6   Email              10000 non-null  int64  
 7   Contract           10000 non-null  object 
 8   Port_modem         10000 non-null  object 
 9   Tablet             10000 non-null  object 
 10  InternetService    10000 non-null  object 
 11  Phone              10000 non-null  object 
 12  Multiple           10000 non-null  object 
 13  OnlineSecurity     10000 non-null  object 
 14  OnlineBackup       10000 non-null  object 
 15  DeviceProtection   10000 non-null  object 
 16  TechSupport        1000

# Data Cleaning

## Duplicates

In [6]:
# check for duplication
df[df.duplicated()]

,Area,Children,Age,Income,Marital,Gender,Email,Contract,Port_modem,Tablet,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,PaymentMethod,Tenure,MonthlyCharge,Bandwidth_GB_Year


## Missing Values

In [7]:
# check for missing values
df.isnull().sum()

Area                 0
Children             0
Age                  0
Income               0
Marital              0
Gender               0
Email                0
Contract             0
Port_modem           0
Tablet               0
InternetService      0
Phone                0
Multiple             0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
PaperlessBilling     0
PaymentMethod        0
Tenure               0
MonthlyCharge        0
Bandwidth_GB_Year    0
dtype: int64

## Outliers

In [8]:
# check for outliers
# import scipy.stats to calculate z-scores
from scipy import stats

outliers = df.select_dtypes(include='number')
outliers = stats.zscore(outliers)
outliers[outliers.abs() >= 3].count()

Children             191
Age                    0
Income               145
Email                 12
Tenure                 0
MonthlyCharge          0
Bandwidth_GB_Year      0
dtype: int64

In [9]:
# children, income, and email features contain outliers
Children_outliers = outliers[outliers.Children.abs() >= 3].index
Income_outliers = outliers[outliers.Income.abs() >= 3].index
Email_outliers = outliers[outliers.Email.abs() >= 3].index

In [10]:
# replace outlier values with median values
df.Children.iloc[Children_outliers] = df.Children.median()
df.Income.iloc[Income_outliers] = df.Income.median()
df.Email.iloc[Email_outliers] = df.Email.median()

# Data Preparation

## Data Description: Categorical Variables

In [11]:
df.select_dtypes(exclude='number').describe()

,Area,Marital,Gender,Contract,Port_modem,Tablet,InternetService,Phone,Multiple,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,PaymentMethod
count,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
unique,3,5,3,3,2,2,3,2,2,2,2,2,2,2,2,2,4
top,Suburban,Divorced,Female,Month-to-month,No,No,Fiber Optic,Yes,No,No,No,No,No,No,No,Yes,Electronic Check
freq,3346,2092,5025,5456,5166,7009,4408,9067,5392,6424,5494,5614,6250,5071,5110,5882,3398


## Data Description: Continuous Variables

In [12]:
df.select_dtypes(include='number').describe()

,Children,Age,Income,Email,Tenure,MonthlyCharge,Bandwidth_GB_Year
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.925000,53.078400,38151.157582,12.022200,34.526188,172.624816,3392.341550
std,1.883383,20.698882,24850.801431,3.004565,26.443063,42.943094,2185.294852
min,0.000000,18.000000,348.670000,3.000000,1.000259,79.978860,155.506715
25%,0.000000,35.000000,19224.717500,10.000000,7.917694,139.979239,1236.470827
50%,1.000000,53.000000,33169.742500,12.000000,35.430507,167.484700,3279.536903
75%,3.000000,71.000000,51669.637500,14.000000,61.479795,200.734725,5586.141370
max,8.000000,89.000000,124025.100000,21.000000,71.999280,290.160419,7158.981530


# Data Transformation

## Data Transformation: Numerical Variables

In [13]:
# transform numerical variables to categorical variables
df.Children = pd.cut(df['Children'].array,bins=[0,1,3,8],labels=['children_1', 'children_2', 'children_3'])
df.Age = pd.cut(df['Age'].array,bins=[18,35,53,71,89],labels=['age_1', 'age_2', 'age_3', 'age_4'])
df.Income = pd.cut(df['Income'].array,bins=[340,19200,33100,51700,124000],labels=['income_1', 'income_2', 'income_3', 'income_4'])
df.Email = pd.cut(df['Email'].array,bins=[3,10,12,14,21],labels=['email_1', 'email_2', 'email_3', 'email_4'])
df.Tenure = pd.cut(df['Tenure'].array,bins=[1,8,35,61,71],labels=['tenure_1', 'tenure_2', 'tenure_3', 'tenure_4'])
df.MonthlyCharge = pd.cut(df['MonthlyCharge'].array,bins=[79,139,167,200,290],labels=['charge_1', 'charge_2', 'charge_3', 'charge_4'])
df.Bandwidth_GB_Year = pd.cut(df['Bandwidth_GB_Year'].array,bins=[150,1230,3280,5590,7160],labels=['bw_1', 'bw_2', 'bw_3', 'bw_4'])                        

## Data Transformation: Categorical Variables

In [14]:
# perform nominal encoding
df = pd.get_dummies(df, columns=['Area', 
                                 'Marital', 
                                 'Gender', 
                                 'Contract', 
                                 'InternetService', 
                                 'PaymentMethod',
                                 'Bandwidth_GB_Year', 
                                 'MonthlyCharge', 
                                 'Tenure', 
                                 'Email', 
                                 'Income', 
                                 'Age', 
                                 'Children'])

In [15]:
# perform ordinal encoding
df.replace('No', 1, inplace=True)
df.replace('Yes', 2, inplace=True)

In [16]:
df.head(5)

,Port_modem,Tablet,Phone,Multiple,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,...,Income_income_2,Income_income_3,Income_income_4,Age_age_1,Age_age_2,Age_age_3,Age_age_4,Children_children_1,Children_children_2,Children_children_3
0,2,2,2,1,2,2,1,1,1,2,...,1,0,0,0,0,1,0,0,0,0
1,1,2,2,2,2,1,1,1,2,2,...,1,0,0,1,0,0,0,1,0,0
2,2,1,2,2,1,1,1,1,1,2,...,0,0,0,0,1,0,0,0,0,1
3,1,1,2,1,2,1,1,1,2,1,...,0,0,0,0,1,0,0,1,0,0
4,2,1,1,1,1,1,1,2,2,1,...,0,1,0,0,0,0,1,0,0,0


## Output

In [17]:
df.to_csv("source_output/churn_prepped1.csv")